**Tema: Algoritmos de Búsqueda**

Los elementos para la representación de un problema de búsqueda son agenda, conjunto de operaciones y estrategia de búsqueda. En las siguientes secciones veremos algunos problemas a resolver y su representación, así como algoritmos de búsqueda.

# Problemas

In [71]:
import numpy as np

## Problema 1: El laberinto

Dado un laberinto representado a traves de una matriz de 0 y 1, donde 0 representa una habitación y 1 representa un muro, una posición inicial I y una posición final F, indique una secuencia (más corta) para llegar de la  habitación I a la F. Los movimientos solo pueden realizarse entre habitaciones contiguas arriba, abajo derecha o izquierda.  

In [1]:
L =  [
      [1,1,1,1,1,1,1,1],
      [1,0,0,0,1,1,0,1],
      [1,0,1,0,1,1,0,1],
      [1,0,1,0,1,1,0,1],
      [1,0,1,0,0,0,0,1],
      [1,0,1,1,0,1,1,1],
      [1,0,0,0,0,0,0,1],
      [1,1,1,1,1,1,1,1],
     ]

print(L);    

[[1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 1, 1, 0, 1], [1, 0, 1, 0, 1, 1, 0, 1], [1, 0, 1, 0, 1, 1, 0, 1], [1, 0, 1, 0, 0, 0, 0, 1], [1, 0, 1, 1, 0, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1]]


Definimos una clase base Configuración que usaremos para este y otros problemas.

In [103]:
class Configuration(object):
      __count  = 0

      def __init__(self):
          Configuration.__count = Configuration.__count+1

          self.id = Configuration.get_count()
          self.previous = None
       
      def getId(self):
          return self.id

      @classmethod     
      def reset_count(cls):
          Configuration.__count = 0

      @classmethod
      def get_count(cls):
          return Configuration.__count   

      def isEqualTo(self,otherConf):
          return true

      def isAValidConfiguration(self):
          return false

      def getSuccessors(self):
          return [] 

      def getSolution(self):
          return [] 

      def setPreviousConfiguration(self,previous):
          self.previous = previous  

      def isAPreviousConfiguration(self,otherConf):
          if (self.previous!=None):
             if self.previous.isEqualTo(otherConf):
               return True 
             else:
               return self.previous.isAPreviousConfiguration(otherConf)
          else:
            return False

      def g(self):
          return 0

      def f(self):
          return 0

      def eval(self):
          return 0                      

Definimos la clase Room que hereda de Configuration. Es decir, definimos qué es una configuración para nuestro problema.

In [104]:
class Room(Configuration):       
      
      __map = None
      __initialRoom = None
      __finalRoom = None

      def __init__(self,row,col):
          super().__init__()
          self.row = row
          self.col = col

      @classmethod
      def set_map(cls, map):
        cls.__map = map

      @classmethod
      def set_initialRoom(cls, initialRoom):
        cls.__initialRoom = initialRoom  

      @classmethod
      def set_finalRoom(cls, finalRoom):
        cls.__finalRoom = finalRoom  

      def up(self):
          room = Room(self.row-1,self.col)
          room.setPreviousConfiguration(self)
          return room         
      
      def down(self):
          room = Room(self.row+1,self.col)
          room.setPreviousConfiguration(self)
          return room

      def left(self):
          room = Room(self.row,self.col-1)
          room.setPreviousConfiguration(self)
          return room  

      def right(self):
          room = Room(self.row,self.col+1)
          room.setPreviousConfiguration(self)
          return room

      def isEqualTo(self,otherConf):
          return self.row == otherConf.row and self.col == otherConf.col
             
      def isAValidConfiguration(self):
          rowSize = len(Room.__map)
          colSize = len(Room.__map[0])
          if self.row >= 0 and self.row < rowSize and \
             self.col >= 0 and self.col < colSize and \
             Room.__map[self.row][self.col]!=1 and \
             not (self.isAPreviousConfiguration(self)):
             
             return True 
          else:
             return False         

      def getSuccessors(self):
          successors = super().getSuccessors()

          conf = self.left()
          if conf.isAValidConfiguration():
             successors.append(conf)

          conf = self.right()
          if conf.isAValidConfiguration():
             successors.append(conf)

          conf = self.up()
          if conf.isAValidConfiguration():
             successors.append(conf)

          conf = self.down()
          if conf.isAValidConfiguration():
             successors.append(conf)   

          return successors

      def getSolution(self):
          solution = []
          currentConf = self
          while currentConf!=None:
             solution.insert(0,currentConf)
             currentConf = currentConf.previous
          return solution

      def g(self):
          value = 0
          currentConf = self
          while currentConf!=None:
             value = value + 1
             currentConf = currentConf.previous
          return value

      def h(self):
          return abs(Room.__finalRoom.row - self.row) + \
                 abs(Room.__finalRoom.col - self.col)

      def eval(self):
          return self.g() + self.h()                    

## Problema 2: 8-puzzle

¿Quién lo define y programa ????????

In [189]:
class PuzzleConfiguration(Configuration):       
      
      __initialConf = None
      __finalConf   = None

      def __init__(self,row,col,board):
          super().__init__()
          self.row = row
          self.col = col
          self.board = board
      
      @classmethod
      def set_initialConf(cls, initialConf):
          cls.__initialConf = initialConf 

      @classmethod
      def set_finalConf(cls, finalConf):
          cls.__finalConf = finalConf

      def cloneBoard(self):
          rowSize = len(self.board)
          colSize = len(self.board[0])

          board = np.zeros((rowSize,colSize),dtype='i')

          for r in range(rowSize): 
              for c in range(colSize):
                  board[r][c] = self.board[r][c]
          return board

      def up(self):          
          if self.row==0:
             return None
          board = self.cloneBoard()
          board[self.row][self.col] = board[self.row-1][self.col]
          board[self.row-1][self.col] = 0
          conf = PuzzleConfiguration(self.row-1,self.col, board)
          conf.setPreviousConfiguration(self)
          return conf         
      
      def down(self):
          rowSize = len(self.board)
          if self.row==rowSize-1:
             return None
          board = self.cloneBoard()          
          board[self.row][self.col] = board[self.row+1][self.col]
          board[self.row+1][self.col] = 0
          conf = PuzzleConfiguration(self.row+1,self.col,board)
          conf.setPreviousConfiguration(self)
          return conf

      def left(self):
          if self.col==0:
             return None
          board = self.cloneBoard()
          board[self.row][self.col] = board[self.row][self.col-1]
          board[self.row][self.col-1] = 0
          conf = PuzzleConfiguration(self.row,self.col-1,board)
          conf.setPreviousConfiguration(self)
          return conf 

      def right(self):
          colSize = len(self.board[0])
          if self.col==colSize-1:
             return None
          board = self.cloneBoard()          
          board[self.row][self.col] = board[self.row][self.col+1]
          board[self.row][self.col+1] = 0
          conf = PuzzleConfiguration(self.row,self.col+1,board)
          conf.setPreviousConfiguration(self)
          return conf  

      def isEqualTo(self,otherConf):
          rowSize = len(self.board)
          colSize = len(self.board[0])
          for r in range(rowSize): 
              for c in range(colSize):
                  if self.board[r][c]!=otherConf.board[r][c]:
                      return False
          return True

      def isAValidConfiguration(self):
          if not (self.isAPreviousConfiguration(self)):             
             return True 
          else:
             return False

      def getSuccessors(self):
          successors = super().getSuccessors()

          conf = self.left()
          if conf!=None and conf.isAValidConfiguration():
             successors.append(conf)

          conf = self.right()
          if conf!=None and conf.isAValidConfiguration():
             successors.append(conf)

          conf = self.up()
          if conf!=None and conf.isAValidConfiguration():
             successors.append(conf)          

          conf = self.down()
          if conf!=None and conf.isAValidConfiguration():
             successors.append(conf)         

          return successors

      def getSolution(self):
          solution = []
          currentConf = self
          while currentConf!=None:
             solution.insert(0,currentConf)
             currentConf = currentConf.previous
          return solution

      def g(self):
          value = 0
          currentConf = self
          while currentConf!=None:
             value = value + 1
             currentConf = currentConf.previous
          return value

      def h1(self):
          rowSize = len(self.board)
          colSize = len(self.board[0])

          value = 0
          for r in range(rowSize): 
              for c in range(colSize):
                  if self.board[r][c]!=self.__finalConf.board[r][c]:
                    value = value + 1
                  
          return value

      def h2(self): 
          rowSize = len(self.board)
          colSize = len(self.board[0])

          selfRows = np.zeros(rowSize*colSize)
          selfCols = np.zeros(rowSize*colSize)
          finalRows = np.zeros(rowSize*colSize)
          finalCols = np.zeros(rowSize*colSize)

          for r in range(rowSize): 
              for c in range(colSize):
                  selfRows[self.board[r][c]] = r
                  selfCols[self.board[r][c]] = c
                  finalRows[self.__finalConf.board[r][c]] = r
                  finalCols[self.__finalConf.board[r][c]] = c

          value = 0
          for i in range(rowSize*colSize):
              value = value + \
                      abs(selfRows[i]-finalRows[i]) + \
                      abs(selfCols[i]-finalCols[i])
          
          return value

      def h3(self):
          pass  

      def h4(self): 
          return abs(self.__finalConf.row - self.row) + \
                 abs(self.__finalConf.col - self.col)            
      
      def h(self):
          return self.h1()



      def eval(self):
          return self.g() + self.h()       

# Estrategias de Búsqueda Ciega

## Búsqueda en amplitud

Entonces programamos algunos de los métodos de busqueda ciega. Empecemos por busqueda a lo ancho (en amplitud).

In [58]:
def breadthFirstSearch(initialConf, finalConf):
    schedule = []
    schedule.append(initialConf)
    while len(schedule)!=0 :
      if schedule[0].isEqualTo(finalConf):
         return schedule[0]
      else:
         first = schedule.pop(0)
         successors = first.getSuccessors()
         for conf in successors:
            schedule.append(conf)  
    return None          

Probemos el método de búsqueda a lo ancho para el problema del **laberinto**.

In [150]:
Room.reset_count()

initialConf = Room(6,1)
finalConf = Room(1,6)

Room.set_map(L)
Room.set_initialRoom(initialConf)
Room.set_finalRoom(finalConf)


resultConf = breadthFirstSearch(initialConf,finalConf)

if resultConf == None:
  print("No existe camino")
else:
  solution = resultConf.getSolution()
  for conf in solution:
    print(conf.row,conf.col)

print(Configuration.get_count()) 

6 1
6 2
6 3
6 4
5 4
4 4
4 5
4 6
3 6
2 6
1 6
106


Probemos el método de búsqueda a lo ancho para el problema del **puzzle**.

In [131]:
initialBoard = [
                [4,1,2],
                [0,5,3],
                [7,8,6],
               ]

finalBoard = [
                [1,2,3],
                [4,5,6],
                [7,8,0],
               ]               
PuzzleConfiguration.reset_count()

initialConf = PuzzleConfiguration(1,0,initialBoard)
finalConf = PuzzleConfiguration(2,2,finalBoard)

PuzzleConfiguration.set_initialConf(initialConf)
PuzzleConfiguration.set_finalConf(finalConf)

resultConf = breadthFirstSearch(initialConf,finalConf)

if resultConf == None:
  print("No existe camino")
else:
  solution = resultConf.getSolution()
  for conf in solution:
    print(conf.row,conf.col)

print(Configuration.get_count()) 

1 0
0 0
0 1
0 2
1 2
2 2
134


## Búsqueda en profundidad

Programemos la búsqueda en profundidad. Podríamos harcelo de la forma tradicional o de forma recursiva.

### Tradicional

In [124]:
def depthFirstSearch(initialConf, finalConf):
    schedule = []
    schedule.append(initialConf)
    while len(schedule)!=0 :
      if schedule[0].isEqualTo(finalConf):
         return schedule[0]
      else:
         first = schedule.pop(0)
         successors = first.getSuccessors()
         for conf in successors:
            schedule.insert(0,conf)
    return None

Probemos el método de búsqueda en profundidad para el problema del **laberinto**.

---



In [ ]:
Room.reset_count()

initialConf = Room(6,1)
finalConf = Room(1,6)

Room.set_map(L)
Room.set_initialRoom(initialConf)
Room.set_finalRoom(finalConf)

resultConf = depthFirstSearch(initialConf,finalConf)

if resultConf == None:
  print("No existe camino")
else:
  solution = resultConf.getSolution()
  for conf in solution:
    print(conf.row,conf.col)

print(Configuration.get_count())

Probemos el método de búsqueda en profundidad para el problema del **puzzle**.

In [ ]:
initialBoard = [
                [4,1,2],
                [0,5,3],
                [7,8,6],
               ]

finalBoard = [
                [1,2,3],
                [4,5,6],
                [7,8,0],
               ]               
PuzzleConfiguration.reset_count()

initialConf = PuzzleConfiguration(1,0,initialBoard)
finalConf = PuzzleConfiguration(2,2,finalBoard)

PuzzleConfiguration.set_initialConf(initialConf)
PuzzleConfiguration.set_finalConf(finalConf)

resultConf = depthFirstSearch(initialConf,finalConf)

if resultConf == None:
  print("No existe camino")
else:
  solution = resultConf.getSolution()
  for conf in solution:
    print(conf.row,conf.col)

print(Configuration.get_count())

### Recursiva

Programemos búsqueda en profundidad pero ahora recursivo.

In [8]:
def depthFirstSearchRecusive(initialConf, finalConf):
    
    if initialConf.isEqualTo(finalConf):
       return initialConf
    else:
       successors = initialConf.getSuccessors()
       for conf in successors:
          resultConf = depthFirstSearchRecusive(conf, finalConf)
          if (resultConf!=None):
            return resultConf
       return None  

Probemos el método de búsqueda en profundidad recursivo para el problema del **laberinto**.

In [ ]:
Room.reset_count()

initialConf = Room(6,1)
finalConf = Room(1,6)

Room.set_map(L)
Room.set_initialRoom(initialConf)
Room.set_finalRoom(finalConf)

resultConf = depthFirstSearchRecusive(initialConf,finalConf)

if resultConf == None:
  print("No existe camino")
else:
  solution = resultConf.getSolution()
  for conf in solution:
    print(conf.row,conf.col)

print(Configuration.get_count())    

Probemos el método de búsqueda en profundidad recursivo para el problema del **puzzle**.

In [ ]:
initialBoard = [
                [4,1,2],
                [0,5,3],
                [7,8,6],
               ]

finalBoard = [
                [1,2,3],
                [4,5,6],
                [7,8,0],
               ]               
PuzzleConfiguration.reset_count()

initialConf = PuzzleConfiguration(1,0,initialBoard)
finalConf = PuzzleConfiguration(2,2,finalBoard)

PuzzleConfiguration.set_initialConf(initialConf)
PuzzleConfiguration.set_finalConf(finalConf)

resultConf = depthFirstSearchRecusive(initialConf,finalConf)

if resultConf == None:
  print("No existe camino")
else:
  solution = resultConf.getSolution()
  for conf in solution:
    print(conf.row,conf.col)

print(Configuration.get_count())

# Estrategias de Búsqueda con Información

Veamos entonces algunas estrategias de búsqueda que usan información.

## Búsqueda primero el mejor

In [167]:
def bestFirstSearch(initialConf, finalConf):
    schedule = []
    schedule.append(initialConf)
    while len(schedule)!=0 :
      if schedule[0].isEqualTo(finalConf):
         return schedule[0]
      else:
         first = schedule.pop(0)
         successors = first.getSuccessors()
         for conf in successors:
            schedule.append(conf)
         schedule.sort(key=lambda x: x.eval(), reverse=False) 
          
    return None

Probemos el método de búsqueda primero el mejor para el problema del **laberinto**.

In [140]:
Room.reset_count()

initialConf = Room(6,1)
finalConf = Room(1,6)

Room.set_map(L)
Room.set_initialRoom(initialConf)
Room.set_finalRoom(finalConf)

resultConf = bestFirstSearch(initialConf,finalConf)

if resultConf == None:
  print("No existe camino")
else:
  solution = resultConf.getSolution()
  for conf in solution:
    print(conf.row,conf.col)

print(Configuration.get_count())

6 1
6 2
6 3
6 4
5 4
4 4
4 5
4 6
3 6
2 6
1 6
78


Probemos el método de búsqueda primero el mejor para el problema del **puzzle**.

In [178]:
initialBoard = [
                [4,1,2],
                [0,5,3],
                [7,8,6],
               ]

finalBoard = [
                [1,2,3],
                [4,5,6],
                [7,8,0],
               ]               
PuzzleConfiguration.reset_count()

initialConf = PuzzleConfiguration(1,0,initialBoard)
finalConf = PuzzleConfiguration(2,2,finalBoard)

PuzzleConfiguration.set_initialConf(initialConf)
PuzzleConfiguration.set_finalConf(finalConf)

resultConf = bestFirstSearch(initialConf,finalConf)

if resultConf == None:
  print("No existe camino")
else:
  solution = resultConf.getSolution()
  for conf in solution:
    print(conf.row,conf.col)

print(Configuration.get_count())

1 0
0 0
0 1
0 2
1 2
2 2
78


## Hill climbing

In [180]:
def hillClimbingSearch(initialConf, finalConf):
    schedule = []
    schedule.append(initialConf)
    while len(schedule)!=0 :
      if schedule[0].isEqualTo(finalConf):
         return schedule[0]
      else:
         first = schedule.pop(0)        
         successors = first.getSuccessors()
         for conf in successors:
            schedule.append(conf)
         schedule.sort(key=lambda x: x.eval(), reverse=False)

         if len(schedule)>0:
            best = schedule[0]
            schedule.clear()
            schedule.append(best)
    return None

Probemos el método de búsqueda Hill Climbing para el problema del **laberinto**.

In [181]:
Room.reset_count()

initialConf = Room(6,1)
finalConf = Room(1,6)

Room.set_map(L)
Room.set_initialRoom(initialConf)
Room.set_finalRoom(finalConf)

resultConf = hillClimbingSearch(initialConf,finalConf)

if resultConf == None:
  print("No existe camino")
else:
  solution = resultConf.getSolution()
  for conf in solution:
    print(conf.row,conf.col)
  
print(Configuration.get_count())    

No existe camino
26


Probemos el método de búsqueda Hill Climbing para el problema del **puzzle**.

In [ ]:
initialBoard = [
                [4,1,2],
                [0,5,3],
                [7,8,6],
               ]

finalBoard = [
                [1,2,3],
                [4,5,6],
                [7,8,0],
               ]               
PuzzleConfiguration.reset_count()

initialConf = PuzzleConfiguration(1,0,initialBoard)
finalConf = PuzzleConfiguration(2,2,finalBoard)

PuzzleConfiguration.set_initialConf(initialConf)
PuzzleConfiguration.set_finalConf(finalConf)

resultConf = hillClimbingSearch(initialConf,finalConf)

if resultConf == None:
  print("No existe camino")
else:
  solution = resultConf.getSolution()
  for conf in solution:
    print(conf.row,conf.col)

print(Configuration.get_count())

## Beam Search

In [191]:
def beamSearch(initialConf, finalConf,k):
    schedule = []
    schedule.append(initialConf)
    while len(schedule)!=0 :
      if schedule[0].isEqualTo(finalConf):
         return schedule[0]
      else:
         first = schedule.pop(0)        
         successors = first.getSuccessors()
         for conf in successors:
            schedule.append(conf)
         schedule.sort(key=lambda x: x.eval(), reverse=False)

         if len(schedule)>0:
            del schedule[k-1:len(schedule)-1]
    return None

Probemos el método de búsqueda beam para el problema del **laberinto**.

In [ ]:
Room.reset_count()

initialConf = Room(6,1)
finalConf = Room(1,6)

Room.set_map(L)
Room.set_initialRoom(initialConf)
Room.set_finalRoom(finalConf)

resultConf = beamSearch(initialConf,finalConf,3)

if resultConf == None:
  print("No existe camino")
else:
  solution = resultConf.getSolution()
  for conf in solution:
    print(conf.row,conf.col)
  
print(Configuration.get_count())    

Probemos el método de búsqueda beam para el problema del **puzzle**.

In [202]:
initialBoard = [
                [4,1,2],
                [0,5,3],
                [7,8,6],
               ]

finalBoard = [
                [1,2,3],
                [4,5,6],
                [7,8,0],
               ]               
PuzzleConfiguration.reset_count()

initialConf = PuzzleConfiguration(1,0,initialBoard)
finalConf = PuzzleConfiguration(2,2,finalBoard)

PuzzleConfiguration.set_initialConf(initialConf)
PuzzleConfiguration.set_finalConf(finalConf)

resultConf = beamSearch(initialConf,finalConf,3)

if resultConf == None:
  print("No existe camino")
else:
  solution = resultConf.getSolution()
  for conf in solution:
    print(conf.row,conf.col)

print(Configuration.get_count())

1 0
0 0
0 1
0 2
1 2
2 2
15
